In [103]:
import pandas as pd
from datetime import datetime

df_list = []

for i in range(1, 31):
    try:
        filename = f'resources/transactions_summary ({i}).csv'
        df = pd.read_csv(filename)
        df_list.append(df)
        print(f'Successfully loaded file {filename}')
        # Perform any data analysis or manipulation you need
    except FileNotFoundError:
        pass

shake_data_df = pd.concat(df_list, ignore_index=True)
shake_data_df['Date'] = pd.to_datetime(shake_data_df['Date']).dt.strftime('%Y-%m-%d')
shake_data_filtered_df = shake_data_df[shake_data_df['Amount debited'] > 1]
shake_data_filtered_df = shake_data_filtered_df[['Amount debited', 'From / To','Date']]
shake_data_filtered_df=shake_data_filtered_df.dropna()
shake_data_filtered_df = shake_data_filtered_df[~shake_data_filtered_df["From / To"].isin(["MASSAGE ADDICT","guevarch@gmail.com","FREEDOM MOBILE", "yansunique@yahoo.ca", "CITY CENTRE BAPTIST", "GOOGLE YouTubePremium"])]
shake_data_filtered_df = shake_data_filtered_df.set_index('Amount debited')
shake_data_filtered_df.to_csv('resources/combined_summary.csv')
shake_data_filtered_df.tail(15)



Successfully loaded file resources/transactions_summary (4).csv
Successfully loaded file resources/transactions_summary (5).csv


,From / To,Date
Amount debited,,
21.53,RCSS 1080,2023-06-04
11.30,DOLLARAMA 840,2023-06-04
86.32,METRO 854,2023-06-04
10.72,Subway 23972,2023-06-06
12.71,DOLLED UP CUPCAKES,2023-06-08
11.63,Subway 23972,2023-06-08
12.65,HARVEYS 0002353 QPS,2023-06-09
109.13,OLD NAVY CA 5462,2023-06-10
114.40,RCSS 1080,2023-06-10
